In [ ]:
from skimage import io
import pandas as pd
import numpy as np
import os
import random

In [ ]:
# prepared resnext model
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from skimage import io, transform

import copy
import os.path
import glob
from torch.utils.data import Dataset, DataLoader
import pandas as pd
plt.ion()   # interactive mode
from matplotlib.pyplot import imshow
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [3, 128, 128]
        # our dimension [3,40,40]
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  # [64, 20, 20]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 10, 10]
            nn.ZeroPad2d(1), # [64,12,12]
    
            nn.Conv2d(64, 128, 3, 1, 1), # [128, 12, 12]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 6, 6]
            nn.ZeroPad2d(1), # [128,8,8]
    
            nn.Conv2d(128, 256, 3, 1, 1), # [256, 8, 8]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 4, 4]
 
            nn.Conv2d(256, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 2, 2]
            
#             nn.Conv2d(512, 512, 3, 1, 1), # [512, 4, 4]
#             nn.BatchNorm2d(512),
#             nn.ReLU(),
#             nn.MaxPool2d(2, 2, 0),       # [512, 2, 2]
        )
        self.cnn = nn.DataParallel(self.cnn)
        self.fc = nn.Sequential(
            nn.Linear(512*2*2, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 4)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

In [ ]:
num_ca = 4
model_use='resnext'
# use resnext model
if model_use=='resnext':
    model_ft = models.resnext50_32x4d(pretrained=False)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs,num_ca)
elif model_use=='diy':
    model_ft = Classifier()

best_model_wts='./layers/resnext_model.p'
model_ft.load_state_dict(torch.load(best_model_wts))
model_ft.eval()

In [ ]:
def predict_category(image):
    # receives (M,N,3) image and returns predicted category
    # print(np.shape(image),end='\t')
    data=torch.tensor(io.imread(image).transpose((2, 0, 1))).view(1,3,60,60)
    outputs = model_ft(data.float())
    # print(outputs)
    _, preds = torch.max(outputs, 1)
    return preds.numpy()[0]

In [ ]:
def predict_pixel(image):
    # receives (M,N,3) image and returns predicted pixel
    print(np.shape(image),end='\t')
    return 1

In [ ]:
def draw_layer(draw=50,folder='layers',home='problem2',resize=0.02,crop_x=60,crop_y=60,cooling_alpha=0.001):
    # create directory
    if not os.path.exists('./%s'%(folder)): os.mkdir('./%s'%(folder))
    if not os.path.exists('./%s/draw'%(folder)): os.mkdir('./%s/draw'%(folder))

    # read label
    layer='L%04d'%(draw)
    mpm_dir='./%s/MPM/Layer%03dto%03d/%s/'%(home,(draw-1)//50*50+1,(draw-1)//50*50+50,layer)
    pixel=[]
    for temp in range(len(os.listdir(mpm_dir))):
        name='%sframe_%04d.bmp'%(mpm_dir,temp+1)
        pixel.append(np.sum(io.imread(name)>150))
    threshold=np.array([10,np.quantile(pixel,.25),np.quantile(pixel,.75)])
    category=[np.sum(temp>threshold) for temp in pixel]

    # read cmd
    cmd=pd.read_csv('./%s/Command_Part1/XYPT_Part01_%s.csv'%(home,layer),header=None)
    left=cmd[0].min()
    right=cmd[0].max()
    down=cmd[1].min()
    up=cmd[1].max()
    mag=1000
    width=int((right-left)*mag*resize)+1
    height=int((up-down)*mag*resize)+1
    power=np.zeros([width,height])
    speed=np.zeros([width,height])
    temporal=np.zeros([width,height])

    current_order=0
    locations=[]
    loc_x=[]
    loc_y=[]
    first_row=cmd[0:1]
    history_x=[first_row[0][0]]*3
    history_y=[first_row[1][0]]*3
    speed_max=-10000
    speed_min=10000
    for line in cmd.iterrows():
        raw_x=line[1][0]
        raw_y=line[1][1]
        history_x.pop()
        history_x.insert(0,raw_x)
        history_y.pop()
        history_y.insert(0,raw_y)
        # calculate speed
        last_speed=((history_x[2]-history_x[0])*(history_x[2]-history_x[0])+(history_y[2]-history_y[0])*(history_y[2]-history_y[0]))**0.5/2e-5
        if current_order>0:
            speed[current_x,current_y]=last_speed
            speed_max=max(last_speed,speed_max)
            speed_min=min(last_speed,speed_min)
        current_order+=1
        current_x=int((raw_x-left)*mag*resize)
        current_y=int((raw_y-down)*mag*resize)
        power[current_x,current_y]=line[1][2]
        temporal[current_x,current_y]=current_order
        if line[1][3]==2:
            locations.append([current_x,current_y])
            loc_x.append(raw_x)
            loc_y.append(raw_y)
    power_min=np.min(power)
    power_max=np.max(power)
    
    # generate photos
    photo_id=0
    for temp in locations:
        if photo_id%100==0: print(photo_id,end='\t')
        center_x,center_y=temp
        power_crop=np.zeros([crop_x,crop_y])
        speed_crop=np.zeros([crop_x,crop_y])
        temporal_crop=np.zeros([crop_x,crop_y])
        for x in range(center_x-int(crop_x*0.5),center_x+int(crop_x*0.5)):
            for y in range(center_y-int(crop_y*0.5),center_y+int(crop_y*0.5)):
                # consider locations in the window with time stamp earlier than center
                if x<0 or x>=width: continue
                if y<0 or y>=height: continue
                if temporal[x,y]==0 or temporal[x,y]>temporal[center_x,center_y]: continue
                # power crop
                original_power=(power[x,y]-power_min)/(power_max-power_min)
                power_crop[x+int(crop_x*0.5)-center_x,y+int(crop_y*0.5)-center_y]=int(original_power*255)
                # speed crop
                original_speed=(speed[x,y]-speed_min)/(speed_max-speed_min)
                speed_crop[x+int(crop_x*0.5)-center_x,y+int(crop_y*0.5)-center_y]=int(original_speed*255)
                # temporal crop
                elapsed_time=temporal[center_x,center_y]-temporal[x,y]
                temporal_crop[x+int(crop_x*0.5)-center_x,y+int(crop_y*0.5)-center_y]=int(np.exp(-elapsed_time*cooling_alpha)*255)
        rgb_image=np.stack((power_crop.astype(np.uint8),speed_crop.astype(np.uint8),temporal_crop.astype(np.uint8)),axis=-1)
        io.imsave('./%s/draw/%d.png'%(folder,photo_id),rgb_image,check_contrast=False)
        photo_id+=1

    pixel_results=[]
    category_results=[]
    for photo in range(photo_id):
        if photo%100==0:print(photo,end='\t')
        img=predict_category('./%s/draw/%d.png'%(folder,photo))
        # pixel_results.append(img)
        # predict category
        category_results.append(img)
    # use quantiles to predict category
    # threshold_results=np.array([10,np.quantile(pixel_results,.25),np.quantile(pixel_results,.75)])
    # category_results=[np.sum(temp>threshold_results) for temp in pixel_results]
    
    # calculate accuracy
    acc=sum(category[temp]==category_results[temp] for temp in range(len(category)))/len(category)
    pd.DataFrame({'x':loc_x,'y':loc_y,
                #   'pixel_label':pixel,
                  'category_label':category,
                #   'pixel_predict':pixel_results,
                  'category_predict':category_results
                  }).to_csv('./%s/heatmap_%d_%0.4f.csv'%(folder,draw,acc),index=0)

In [ ]:
for draw in range(1,251,10):
    draw_layer(draw=draw)